In [2]:
%cd drive/MyDrive/Colab\ Notebooks/GPU/rodinia_3.1

/content/drive/MyDrive/Colab Notebooks/GPU/rodinia_3.1


In [30]:
!nvidia-smi

Wed Nov 30 19:06:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!ls

bin	cuda  LICENSE	opencl	others	result.txt
common	data  Makefile	openmp	README


In [16]:
!cat /proc/cpuinfo && nproc --all

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0xffffffff
cpu MHz		: 2000.174
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmi

# Hotspot

### CUDA

In [5]:
!cd cuda/hotspot3D/ && make

/usr/local/cuda/bin/nvcc  -g -G  3D.cu -o 3D  -I/usr/local/cuda/include -L/usr/local/cuda/lib64 


In [7]:
!./cuda/hotspot3D/3D 512 8 200 data/hotspot3D/power_512x8 data/hotspot3D/temp_512x8 cuda/hotspot3D/output.out

Time: 0.0836850032 (s)
Accuracy: 4.615587e-05


### OMP

In [9]:
!cd openmp/hotspot3D/ && make

gcc -o 3D -g -fopenmp -O3 3D.c -lm 
3D.c: In function ‘computeTempOMP’:
3D.c:157:40: warning: implicit declaration of function ‘omp_get_num_threads’ [-Wimplicit-function-declaration]
         printf("%d threads running\n", omp_get_num_threads());
                                        ^~~~~~~~~~~~~~~~~~~


In [10]:
!./openmp/hotspot3D/3D 512 8 200 data/hotspot3D/power_512x8 data/hotspot3D/temp_512x8 cuda/hotspot3D/output.out

2 threads running
Time: 2.411 (s)
Accuracy: 4.615587e-05


# Kmeans

### CUDA

In [34]:
!cd cuda/kmeans/ && make

gcc -g -fopenmp -O2  kmeans.c -c
kmeans.c: In function ‘setup’:
kmeans.c:131:16: warning: implicit declaration of function ‘getopt’; did you mean ‘setup’? [-Wimplicit-function-declaration]
   while ( (opt=getopt(argc,argv,"i:t:m:n:l:bro"))!= EOF) {
                ^~~~~~
                setup
kmeans.c:167:9: warning: implicit declaration of function ‘read’; did you mean ‘fread’? [-Wimplicit-function-declaration]
         read(infile, &npoints,   sizeof(int));
         ^~~~
         fread
kmeans.c:179:9: warning: implicit declaration of function ‘close’; did you mean ‘pclose’? [-Wimplicit-function-declaration]
         close(infile);
         ^~~~~
         pclose
gcc -g -fopenmp -O2  cluster.o getopt.o kmeans.o kmeans_clustering.o kmeans_cuda.o rmse.o -o kmeans -L/usr/local/cuda/lib64 -lcuda -lcudart -lm


In [35]:
!./cuda/kmeans/kmeans -o -i data/kmeans/kdd_cup 


I/O completed

Number of objects: 494020
Number of features: 34
iterated 2 times

================= Centroid Coordinates =================
0: 47.98 1617.80 868.56 0.01 0.00 0.03 0.00 0.01 0.00 0.00 0.01 0.00 0.00 0.00 0.00 332.67 293.24 0.18 0.18 0.06 0.06 0.79 0.02 0.03 233.22 189.18 0.75 0.03 0.60 0.01 0.18 0.18 0.06 0.06

1: 0.00 239.00 486.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 8.00 8.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 19.00 19.00 1.00 0.00 0.05 0.00 0.00 0.00 0.00 0.00

2: 0.00 235.00 1337.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 8.00 8.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 29.00 29.00 1.00 0.00 0.03 0.00 0.00 0.00 0.00 0.00

3: 0.00 219.00 1337.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 6.00 6.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 39.00 39.00 1.00 0.00 0.03 0.00 0.00 0.00 0.00 0.00

4: 0.00 217.00 2032.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 6.00 6.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 49

### OMP

In [31]:
!cd openmp/kmeans/kmeans_openmp/ && make

gcc -g -fopenmp -O2  kmeans.c -c
kmeans.c: In function ‘main’:
kmeans.c:152:9: warning: implicit declaration of function ‘read’; did you mean ‘fread’? [-Wimplicit-function-declaration]
         read(infile, &numObjects,    sizeof(int));
         ^~~~
         fread
kmeans.c:165:9: warning: implicit declaration of function ‘close’; did you mean ‘pclose’? [-Wimplicit-function-declaration]
         close(infile);
         ^~~~~
         pclose
gcc -g -fopenmp -O2  cluster.o getopt.o kmeans.o kmeans_clustering.o  -o kmeans


In [33]:
!./openmp/kmeans/kmeans_openmp/kmeans -n 2 -i data/kmeans/kdd_cup 

I/O completed
num of threads = 2
number of Clusters 5
number of Attributes 34

Cluster Centers Output
The first number is cluster number and the following data is arribute value

0: 3664.94 4313428.50 0.00 0.00 0.00 1.95 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.21 1.50 0.01 0.01 0.01 0.01 0.98 0.03 0.04 53.84 42.38 0.74 0.04 0.77 0.08 0.01 0.02 0.01 0.00 

1: 47.43 632.56 494.05 0.01 0.00 0.03 0.00 0.00 0.00 0.00 0.01 0.00 0.00 0.00 0.00 334.35 294.75 0.18 0.18 0.06 0.06 0.79 0.02 0.03 233.32 189.07 0.75 0.03 0.60 0.01 0.18 0.18 0.06 0.06 

2: 34.67 54557.97 21945.31 0.00 0.00 1.72 0.00 1.74 0.00 0.00 0.99 0.00 0.00 0.01 0.00 3.66 4.02 0.00 0.00 0.03 0.08 1.00 0.00 0.11 195.45 196.63 0.95 0.01 0.03 0.00 0.00 0.00 0.04 0.04 

3: 44.29 533.96 1472426.12 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.04 1.07 0.00 0.00 0.00 0.00 1.00 0.00 0.04 150.21 37.46 0.30 0.03 0.11 0.03 0.00 0.00 0.00 0.00 

4: 552.28 108.94 5009689.50 0.00 0.00 0.00 0.17 1.00 0.06 0.11 0.50 0.00

# LavaMD

### CUDA

In [40]:
!cd cuda/lavaMD/ && make KERNEL_DIM="-DRD_WG_SIZE_0=128"

/usr/local/cuda/bin/nvcc -DRD_WG_SIZE_0=128						./kernel/kernel_gpu_cuda_wrapper.cu \
								-c \
								-o ./kernel/kernel_gpu_cuda_wrapper.o \
								-O3 \
								-arch sm_75
./kernel/kernel_gpu_cuda_wrapper.cu: In function ‘void kernel_gpu_cuda_wrapper(par_str, dim_str, box_str*, FOUR_VECTOR*, double*, FOUR_VECTOR*)’:
./kernel/kernel_gpu_cuda_wrapper.cu:66:23: warning: ‘cudaError_t cudaThreadSynchronize()’ is deprecated [-Wdeprecated-declarations]
  cudaThreadSynchronize();
                       ^
/usr/local/cuda/bin/../targets/x86_64-linux/include/cuda_runtime_api.h:1011:46: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                              ^~~~~~~~~~~~~~~~~~~~~
./kernel/kernel_gpu_cuda_wrapper.cu:196:23: warning: ‘cudaError_t cudaThreadSynchronize()’ is deprecated [-Wdeprecated-declarations]
  cudaThreadSynchronize();
                       ^
/usr/local/cuda/bin/../targets/x86_64-linux/inclu

In [42]:
!./cuda/lavaMD/lavaMD -boxes1d 10

thread block size of kernel = 128 
Configuration used: boxes1d = 10
Time spent in different stages of GPU_CUDA KERNEL:
 0.190824002028 s, 46.938438415527 % : GPU: SET DEVICE / DRIVER INIT
 0.000452999986 s,  0.111427880824 % : GPU MEM: ALO
 0.001961000031 s,  0.482362180948 % : GPU MEM: COPY IN
 0.211918994784 s, 52.127338409424 % : GPU: KERNEL
 0.000783000025 s,  0.192600503564 % : GPU MEM: COPY OUT
 0.000600999978 s,  0.147832572460 % : GPU MEM: FRE
Total time:
0.406540989876 s


### OMP

In [43]:
!cd openmp/lavaMD/ && make

gcc	 main.c \
		-c \
		-o main.o \
		-O3
main.c: In function ‘main’:
main.c:95:6: warning: implicit declaration of function ‘strcmp’ [-Wimplicit-function-declaration]
   if(strcmp(argv[dim_cpu.cur_arg], "-cores")==0){
      ^~~~~~
main.c:253:8: warning: implicit declaration of function ‘time’; did you mean ‘true’? [-Wimplicit-function-declaration]
  srand(time(NULL));
        ^~~~
        true
gcc				./kernel/kernel_cpu.c \
					-c \
					-o ./kernel/kernel_cpu.o \
					-O3 \
					-fopenmp
./kernel/kernel_cpu.c: In function ‘kernel_cpu’:
./kernel/kernel_cpu.c:206:31: warning: unknown conversion type character ‘:’ in format [-Wformat=]
  printf("%15.12f s, %15.12f % : CPU/MCPU: VARIABLES\n",    (float) (time1-time0) / 1000000, (float) (time1-time0) / (float) (time4-time0) * 100);
                               ^
./kernel/kernel_cpu.c:207:31: warning: unknown conversion type character ‘:’ in format [-Wformat=]
  printf("%15.12f s, %15.12f % : MCPU: SET DEVICE\n",     (float) (time2-time1

In [47]:
!./openmp/lavaMD/lavaMD -cores 2 -boxes1d 10

Configuration used: cores = 2, boxes1d = 10
Time spent in different stages of CPU/MCPU KERNEL:
 0.000000000000 s,  0.000000000000 % : CPU/MCPU: VARIABLES
 0.000011000000 s,  0.000161862801 % : MCPU: SET DEVICE
 0.000000000000 s,  0.000000000000 % : CPU/MCPU: INPUTS
 6.795867919922 s, 99.999839782715 % : CPU/MCPU: KERNEL
Total time:
6.795878887177 s
